<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_02_3_llm_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 2: Code Generation**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 2 Material

* Part 2.1: Prompting for Code Generation [[Video]](https://www.youtube.com/watch?v=HVId6kYKKgQ) [[Notebook]](t81_559_class_02_1_dev.ipynb)
* Part 2.2: Handling Revision Prompts [[Video]](https://www.youtube.com/watch?v=APpV46tplXA) [[Notebook]](t81_559_class_02_2_multi_prompt.ipynb)
* **Part 2.3: Using a LLM to Help Debug** [[Video]](https://www.youtube.com/watch?v=VPqSNb38QK0) [[Notebook]](t81_559_class_02_3_llm_debug.ipynb)
* Part 2.4: Tracking Prompts in Software Development [[Video]](https://www.youtube.com/watch?v=oUFUuYfvXZU) [[Notebook]](t81_559_class_02_4_software_eng.ipynb)
* Part 2.5: Limits of LLM Code Generation [[Video]](https://www.youtube.com/watch?v=dKtRI0LZSyY) [[Notebook]](t81_559_class_02_5_code_gen_limits.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.2 MB/s eta 0:00:00


# 2.3: Using a LLM to Help Debug

LLMs can help you debug both the code you create and the code you generate to fulfill your requests. In this part, you will see how to use an LLM as an assistant to help debug a Python program.

## Conversational Code Generation

We will continue to use the conversational code generation function provided in Module 2.2.



In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from IPython.display import display_markdown

MODEL = "gpt-5-mini"
SYSTEM_TEMPLATE = """The following is a friendly conversation between a human and an
AI to generate Python code. If you have notes about the code, place them before
the code. Any notes about execution should follow the code. If you do mix any
notes with the code, make them comments. Add proper comments to the code.
Sort imports and follow PEP 8 formatting.
"""

_histories = {}
def _get_history(session_id: str):
    return _histories.setdefault(session_id, InMemoryChatMessageHistory())

def _build_chain():
    llm = ChatOpenAI(model=MODEL, temperature=0.0, n=1)
    prompt = ChatPromptTemplate.from_messages([
        ("system", SYSTEM_TEMPLATE),
        MessagesPlaceholder("history"),
        ("human", "{input}")
    ])
    return prompt | llm | StrOutputParser()

def start_conversation(session_id: str = "default"):
    base = _build_chain()
    with_history = RunnableWithMessageHistory(
        base,
        lambda sid: _get_history(sid),
        input_messages_key="input",
        history_messages_key="history",
    )
    # Return a runnable pre-bound to the session id
    return with_history.with_config(configurable={"session_id": session_id})

def generate_code(conversation, prompt: str):
    print("Model response:")
    result = conversation.invoke({"input": prompt})
    display_markdown(result, raw=True)
    return result

# Usage:
# conversation = start_conversation()  # or start_conversation("my-session")
# generate_code(conversation, "Write a function to compute Levenshtein distance.")


## A Buggy Pi Approximator

To see an example of how you can make use of LLM-enabled debugging, consider the following code to use the [Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) method to estimate [Pi](https://en.wikipedia.org/wiki/Pi). We need to fix several issues with this code. We can request the LLM to help us debug. This code, when executed, produces the following error:

```
NameError: name 'xrange' is not defined
```

In [3]:
import random

def monte_carlo_pi(num_samples):
    inside_circle = 0

    for _ in xrange(num_samples):
        x, y = random.random(), random.random()  # Generate random point (x, y)
        if x*2 + y*2 <= 1:
            inside_circle += 1  # Check if the point is inside the quarter circle

    pi_approximation = 4 * inside_circle / num_samples  # Calculate approximation of Pi
    return pi_approximation

# Example usage
num_samples = 1000000  # Number of random points to generate
approximated_pi = monte_carlo_pi(num_samples)
print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")

NameError: name 'xrange' is not defined

When we ask the LLM to help us debug this code, we should provide as much detail as possible. I usually like to produce a prompt in the following format:

```
I am trying to debug the following code:

... provide code here...

However, I am getting the following error:

... add the error here, provide stack trace ...

```

In [4]:
conversation = start_conversation()
generate_code(conversation, """
I am trying to debug the following code:

import random

def monte_carlo_pi(num_samples):
    inside_circle = 0

    for _ in xrange(num_samples):
        x, y = random.random(), random.random()  # Generate random point (x, y)
        if x*2 + y*2 <= 1:
            inside_circle += 1  # Check if the point is inside the quarter circle

    pi_approximation = 4 * inside_circle / num_samples  # Calculate approximation of Pi
    return pi_approximation

# Example usage
num_samples = 1000000  # Number of random points to generate
approximated_pi = monte_carlo_pi(num_samples)
print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")

However, I am getting the following error:

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-10-c7b4356f1718> in <cell line: 16>()
     14 # Example usage
     15 num_samples = 1000000  # Number of random points to generate
---> 16 approximated_pi = monte_carlo_pi(num_samples)
     17 print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")

<ipython-input-10-c7b4356f1718> in monte_carlo_pi(num_samples)
      4     inside_circle = 0
      5
----> 6     for _ in xrange(num_samples):
      7         x, y = random.random(), random.random()  # Generate random point (x, y)
      8         if x*2 + y*2 <= 1:

NameError: name 'xrange' is not defined

""")

Model response:


Notes about the bug and fixes
- Problem 1: xrange is a Python 2 function; in Python 3 you should use range().
- Problem 2: The inside-circle test used x*2 + y*2 which is arithmetic (2*x + 2*y). It should be x**2 + y**2 to test distance from origin.
- I also added basic input validation and a small main guard for example usage.

# Corrected code

```python
# Monte Carlo Pi approximation
# - Uses random sampling in the unit square [0, 1) x [0, 1)
# - Counts points inside the quarter circle of radius 1 (x**2 + y**2 <= 1)
# - Pi is approximated as 4 * (points_inside / total_points)

import math
import random


def monte_carlo_pi(num_samples: int) -> float:
    """Approximate Pi using the Monte Carlo method.

    Args:
        num_samples: Number of random samples to draw. Must be a positive integer.

    Returns:
        Approximation of Pi as a float.

    Raises:
        ValueError: If num_samples is not a positive integer.
    """
    if not isinstance(num_samples, int) or num_samples <= 0:
        raise ValueError("num_samples must be a positive integer")

    inside_circle = 0

    # Use range() in Python 3. Generate random (x, y) in [0, 1) and
    # test whether x**2 + y**2 <= 1 (quarter circle of radius 1).
    for _ in range(num_samples):
        x = random.random()
        y = random.random()
        if x * x + y * y <= 1.0:
            inside_circle += 1

    # Multiply by 4 because we're sampling a unit square covering a quarter circle.
    pi_approximation = 4.0 * inside_circle / num_samples
    return pi_approximation


if __name__ == "__main__":
    # Example usage
    num_samples = 1_000_000  # Number of random points to generate
    approximated_pi = monte_carlo_pi(num_samples)
    print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")
    print(f"math.pi = {math.pi}")
```

Notes about execution
- Run with Python 3 (python3 script.py or in an interactive environment).
- Result is a stochastic approximation; increasing num_samples improves accuracy but costs more time.
- For much faster and more accurate results, consider vectorized sampling with numpy (e.g., generate arrays of random numbers and compute distances in bulk).

'Notes about the bug and fixes\n- Problem 1: xrange is a Python 2 function; in Python 3 you should use range().\n- Problem 2: The inside-circle test used x*2 + y*2 which is arithmetic (2*x + 2*y). It should be x**2 + y**2 to test distance from origin.\n- I also added basic input validation and a small main guard for example usage.\n\n# Corrected code\n\n```python\n# Monte Carlo Pi approximation\n# - Uses random sampling in the unit square [0, 1) x [0, 1)\n# - Counts points inside the quarter circle of radius 1 (x**2 + y**2 <= 1)\n# - Pi is approximated as 4 * (points_inside / total_points)\n\nimport math\nimport random\n\n\ndef monte_carlo_pi(num_samples: int) -> float:\n    """Approximate Pi using the Monte Carlo method.\n\n    Args:\n        num_samples: Number of random samples to draw. Must be a positive integer.\n\n    Returns:\n        Approximation of Pi as a float.\n\n    Raises:\n        ValueError: If num_samples is not a positive integer.\n    """\n    if not isinstance(num_

In this case, the LLM decided to be an overachiever because I only asked it about the specific error I was getting. However, the LLM provided me with two issues, one of which was the error I encountered. The LLM identified these two issues:

* It looks like you're using Python 3, where xrange has been replaced by range.

* Also, there's a mistake in the condition to check if the point is inside the quarter circle. It should be ```x ** 2 + y ** 2 <= 1``` instead ```of x*2 + y*2 <= 1```.

The LLM also provided a corrected code for me to copy/paste.

## Testing the Corrected Code

Now, we can test the corrected code and see that it works properly.

In [5]:
import random

def monte_carlo_pi(num_samples):
    """
    Estimate the value of Pi using the Monte Carlo method.

    Args:
    num_samples (int): Number of random samples to generate.

    Returns:
    float: Approximated value of Pi.
    """
    inside_circle = 0

    for _ in range(num_samples):  # Use range instead of xrange for Python 3
        x, y = random.random(), random.random()  # Generate random point (x, y)
        if x**2 + y**2 <= 1:  # Correct formula to check if inside the quarter circle
            inside_circle += 1

    pi_approximation = 4 * inside_circle / num_samples  # Calculate approximation of Pi
    return pi_approximation

# Example usage
num_samples = 1000000  # Number of random points to generate
approximated_pi = monte_carlo_pi(num_samples)
print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")

Approximated Pi with 1000000 samples: 3.140488


## LLMs Explaining Code

LLMs are also very adept at explaining code. As you work through this course, you will see that the assignments use a submission function I named "submit." This submission function uses HTTP and API calling techniques that are not covered by this course. However, if you are interested in what the "submit" function does, you can ask the LLM.

In [6]:
# Start a new conversation
conversation = start_conversation()
generate_code(conversation, """
Could you please explain what the following code does?

import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))
""")

Model response:


Notes
- Below I explain what the function does step by step, point out bugs/oddities, and suggest fixes and improvements.
- I include a tiny corrected code snippet to fix the main bug (BytesIO usage) and one suggested improvement (use isinstance). The snippet is short and PEP 8 compliant.

What this code does (high level)
- Provides a submit(data, key, no, source_file=None) function intended to submit an assignment to a remote API.
- Accepts a list (data) of pandas DataFrames and/or PIL Images, encodes them into base64, attaches the student's source file (a .py or .ipynb) base64-encoded, and POSTs the payload to https://api.heatonresearch.com/assignment-submit with the API key in the header.

Parameters
- data: list of items. The function expects items to be either:
  - PIL.Image.Image objects (images), or
  - pandas.DataFrame objects (CSV payload).
  Other item types are ignored.
- key: string API key included in the HTTP header 'x-api-key'.
- no: assignment number (the code expects to format suffix '_class{no}' and check that string is in the source_file name).
- source_file: path to the student's source file (.py or .ipynb). If running in a script, __file__ will be used unless you’re in a Jupyter notebook (then you must pass source_file explicitly because __file__ is not defined).

Step-by-step behavior
1. If source_file is None and the global __file__ is not present, the function raises an Exception telling you to provide a filename (this is to support notebooks).
2. If source_file is None but __file__ exists, source_file becomes __file__.
3. It builds a suffix string '_class{no}' and enforces that suffix appears in the provided source_file path; if not, raises an Exception. This enforces filename convention (e.g., file must contain "_class1").
4. It opens the source_file in binary mode, reads it, and base64-encodes the entire file contents into encoded_python.
5. It checks the file extension and requires it to be .ipynb or .py, otherwise raises an Exception.
6. It builds a payload list. For each item in data:
   - If the item is a PIL image, it saves the image into a BytesIO buffer as PNG, base64-encodes the PNG bytes, and appends a dict {'PNG': <base64-string>} to payload.
   - If the item is a pandas DataFrame, it converts it to CSV text with to_csv(index=False), encodes that to ASCII bytes, base64-encodes, and appends {'CSV': <base64-string>} to payload.
7. It POSTs a JSON body containing payload, assignment number, extension, and the base64-encoded source file under key 'py' to the remote endpoint. The API key is sent via an HTTP header 'x-api-key'.
8. If the response status code is 200 it prints "Success: <response-text>", otherwise prints "Failure: <response-text>".

Main bug / issues in the provided code
- Name/usage bug: The code uses BytesIO() but does not import it or reference it from io. The file had import io at top but uses BytesIO directly (should be io.BytesIO() or from io import BytesIO). This will raise NameError at runtime.
- Type checks: The code uses `type(item) is PIL.Image.Image` and `type(item) is pd.core.frame.DataFrame`. Using type(...) is brittle; use isinstance(...) instead to allow subclasses and to be more robust (and avoid referencing pd internals).
- Unused imports: numpy is imported but not used.
- Security / privacy: The function reads and uploads the full source file and sends it to the remote server — be careful not to leak secrets (like local API keys) in the file or in the key variable logged or printed.
- Size limits: Large files or large data payloads will produce very large base64 strings; there is no check against payload size.
- Error handling: The requests.post call has no try/except; connection errors or timeouts will raise exceptions. No timeout is set.
- Only two item types are handled; other types are silently ignored (no warning).
- The function posts the encoded source file under key 'py' and also includes ext. The API’s exact expected JSON shape should be confirmed with the API docs.

Suggested fixes and improvements
- Fix BytesIO usage: use io.BytesIO() or from io import BytesIO.
- Use isinstance(item, PIL.Image.Image) and isinstance(item, pd.DataFrame).
- Add try/except around requests.post and specify a timeout.
- Optionally, validate/limit payload size before sending.
- Consider returning the parsed JSON result or status code rather than just printing.
- If you expect other types (e.g., numpy arrays), convert/handle them explicitly or raise a warning/error for unsupported types.

Small corrected snippet (fixes BytesIO + uses isinstance)
Notes about the snippet:
- Shows only the relevant payload construction block, sorted imports and PEP 8 style.
- Uses io.BytesIO and isinstance.

import base64
import io
import os

import pandas as pd
import PIL.Image
import requests

# Build payload from data list, encoding images and dataframes as base64.
payload = []
for item in data:
    if isinstance(item, PIL.Image.Image):
        buffered = io.BytesIO()
        item.save(buffered, format="PNG")
        payload.append({"PNG": base64.b64encode(buffered.getvalue()).decode("ascii")})
    elif isinstance(item, pd.DataFrame):
        csv_bytes = item.to_csv(index=False).encode("ascii")
        payload.append({"CSV": base64.b64encode(csv_bytes).decode("ascii")})

Execution notes
- Fix the BytesIO bug and add safe error handling before running.
- Run in Python 3.
- Be cautious about uploading sensitive files.
- If you want, I can provide a fully revised version of the whole submit function with improved error handling, logging, and type checks.

'Notes\n- Below I explain what the function does step by step, point out bugs/oddities, and suggest fixes and improvements.\n- I include a tiny corrected code snippet to fix the main bug (BytesIO usage) and one suggested improvement (use isinstance). The snippet is short and PEP 8 compliant.\n\nWhat this code does (high level)\n- Provides a submit(data, key, no, source_file=None) function intended to submit an assignment to a remote API.\n- Accepts a list (data) of pandas DataFrames and/or PIL Images, encodes them into base64, attaches the student\'s source file (a .py or .ipynb) base64-encoded, and POSTs the payload to https://api.heatonresearch.com/assignment-submit with the API key in the header.\n\nParameters\n- data: list of items. The function expects items to be either:\n  - PIL.Image.Image objects (images), or\n  - pandas.DataFrame objects (CSV payload).\n  Other item types are ignored.\n- key: string API key included in the HTTP header \'x-api-key\'.\n- no: assignment number (

As you can see, the LLM explained my "submit" function.

## Improving Code with LLMs

You can also request that a LLM improve your code. You can mention specific improvements you seek, such as removing unused or redundant imports, sorting the imports, and adhering to PEP-8 for your code formatting. In the following code, I request that the LLM improve my submit function.

In [7]:
conversation = start_conversation()
generate_code(conversation, """
Could you please suggest and implement any improvements to the following code?

import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))
""")

Model response:


Notes about improvements implemented
- Fixes: use io.BytesIO (was missing), use isinstance(...) instead of type(...) checks.
- Robustness: validate inputs (types, file existence, assignment suffix), support numpy arrays by converting to images or CSVs, and report ignored/unsupported items.
- Reliability: add timeout and try/except around network call, return structured result instead of only printing, and log warnings.
- Safety: warn when source file or payload is large.
- Style: sorted imports, type hints, docstring, clear comments, and PEP 8 formatting.

# Improved submit function

```python
# Improved assignment submit utility
# - Encodes provided pandas DataFrames, PIL Images, or numpy arrays as base64 payloads
# - Attaches the source .py or .ipynb file (must contain "_class{no}" in filename)
# - Posts the payload to the remote submission endpoint with error handling and timeout

import base64
import io
import logging
import os
from typing import Any, Dict, List, Optional, Union

import numpy as np
import pandas as pd
import requests
from PIL import Image

# Configure a simple logger for warnings/info; client code can reconfigure as needed.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def submit(
    data: List[Union[pd.DataFrame, Image.Image, np.ndarray]],
    key: str,
    no: int,
    source_file: Optional[str] = None,
    timeout: int = 10,
    max_source_bytes: int = 10 * 1024 * 1024,
) -> Dict[str, Any]:
    """Submit assignment data and source file to the remote API.

    Args:
        data: List of items to submit. Supported types:
            - pandas.DataFrame -> encoded as CSV
            - PIL.Image.Image -> encoded as PNG
            - numpy.ndarray -> if 2D/3D converted to image (PNG), otherwise converted to CSV
        key: API key string to send in 'x-api-key' header.
        no: Assignment number. Source filename must contain "_class{no}".
        source_file: Path to the .py or .ipynb file to upload. If None, uses __file__ if available.
        timeout: Seconds to wait for the HTTP request.
        max_source_bytes: Warn if source file (in bytes) exceeds this threshold.

    Returns:
        A dict with keys:
            - 'success': bool
            - 'status_code': int or None
            - 'response_text': str (server response or error message)
            - 'ignored_count': int (number of unsupported items)
            - 'payload_item_count': int
    """
    # Validate key and assignment number
    if not isinstance(key, str) or not key.strip():
        raise ValueError("key must be a non-empty string")

    if not isinstance(no, int) or no <= 0:
        raise ValueError("no (assignment number) must be a positive integer")

    # Determine source_file
    if source_file is None:
        if "__file__" not in globals():
            raise Exception(
                "Must specify source_file when running in a Jupyter notebook or interactive session."
            )
        source_file = __file__

    if not os.path.isfile(source_file):
        raise FileNotFoundError(f"Source file not found: {source_file}")

    # Enforce filename suffix convention
    suffix = f"_class{no}"
    if suffix not in os.path.basename(source_file):
        raise Exception(f"Filename must contain the suffix '{suffix}'")

    # Read and encode the source file
    with open(source_file, "rb") as f:
        source_bytes = f.read()

    if len(source_bytes) > max_source_bytes:
        logger.warning(
            "Source file size is %d bytes which exceeds the warning threshold of %d bytes.",
            len(source_bytes),
            max_source_bytes,
        )

    encoded_python = base64.b64encode(source_bytes).decode("ascii")

    # Check extension
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in [".ipynb", ".py"]:
        raise Exception(f"Source file extension {ext} is not supported; must be .py or .ipynb")

    # Build payload: list of dicts with keys 'PNG' or 'CSV'
    payload: List[Dict[str, str]] = []
    ignored_count = 0

    for idx, item in enumerate(data):
        # PIL images
        if isinstance(item, Image.Image):
            buf = io.BytesIO()
            # Save as PNG into buffer
            item.save(buf, format="PNG")
            png_bytes = buf.getvalue()
            payload.append({"PNG": base64.b64encode(png_bytes).decode("ascii")})
            buf.close()
            continue

        # pandas DataFrame
        if isinstance(item, pd.DataFrame):
            csv_bytes = item.to_csv(index=False).encode("utf-8")
            payload.append({"CSV": base64.b64encode(csv_bytes).decode("ascii")})
            continue

        # numpy array: convert to image if 2D/3D, else convert to CSV-like text
        if isinstance(item, np.ndarray):
            if item.ndim in (2, 3):
                # Normalize or convert dtype for imagery if necessary
                arr = item
                # If boolean or float, try to convert to uint8 appropriately
                if arr.dtype == np.bool_:
                    arr = (arr.astype(np.uint8) * 255)
                elif np.issubdtype(arr.dtype, np.floating):
                    # scale floats assumed in [0,1] to [0,255], clip to be safe
                    arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
                else:
                    # if integer type, ensure uint8 or convert
                    arr = arr.astype(np.uint8)

                # If grayscale 2D, create Image from mode 'L'
                try:
                    img = Image.fromarray(arr)
                except Exception as e:
                    logger.warning("Failed to convert numpy array at index %d to image: %s", idx, e)
                    # Fall back to CSV conversion below
                else:
                    buf = io.BytesIO()
                    img.save(buf, format="PNG")
                    payload.append({"PNG": base64.b64encode(buf.getvalue()).decode("ascii")})
                    buf.close()
                    continue

            # Fallback: convert numpy array to CSV (1D or failed image)
            try:
                df = pd.DataFrame(item)
                csv_bytes = df.to_csv(index=False).encode("utf-8")
                payload.append({"CSV": base64.b64encode(csv_bytes).decode("ascii")})
            except Exception as e:
                logger.warning("Failed to convert numpy array at index %d to CSV: %s", idx, e)
                ignored_count += 1
            continue

        # Unsupported item type
        logger.warning("Ignoring unsupported data item at index %d (type %s)", idx, type(item))
        ignored_count += 1

    # Perform POST request with timeout and error handling
    url = "https://api.heatonresearch.com/assignment-submit"
    headers = {"x-api-key": key}
    body = {"payload": payload, "assignment": no, "ext": ext, "py": encoded_python}

    try:
        resp = requests.post(url, headers=headers, json=body, timeout=timeout)
    except requests.RequestException as exc:
        logger.error("Network error while submitting assignment: %s", exc)
        return {
            "success": False,
            "status_code": None,
            "response_text": str(exc),
            "ignored_count": ignored_count,
            "payload_item_count": len(payload),
        }

    # Return structured information
    success = resp.status_code == 200
    if success:
        logger.info("Submission successful: %s", resp.text)
    else:
        logger.error("Submission failed (status %d): %s", resp.status_code, resp.text)

    return {
        "success": success,
        "status_code": resp.status_code,
        "response_text": resp.text,
        "ignored_count": ignored_count,
        "payload_item_count": len(payload),
    }
```

Execution notes
- Run under Python 3.6+ (typing and f-strings used).
- If you want only console prints, you can reconfigure the logger with logging.basicConfig(level=logging.INFO).
- The function returns a dict describing the outcome instead of only printing; client code can inspect this to decide further action.
- If you prefer stricter behavior, change logger.warning calls into raised Exceptions for unsupported items.
- If you want additional features (e.g., payload size limit enforcement, progress callbacks, authentication via other methods), tell me and I can add them.

'Notes about improvements implemented\n- Fixes: use io.BytesIO (was missing), use isinstance(...) instead of type(...) checks.\n- Robustness: validate inputs (types, file existence, assignment suffix), support numpy arrays by converting to images or CSVs, and report ignored/unsupported items.\n- Reliability: add timeout and try/except around network call, return structured result instead of only printing, and log warnings.\n- Safety: warn when source file or payload is large.\n- Style: sorted imports, type hints, docstring, clear comments, and PEP 8 formatting.\n\n# Improved submit function\n\n```python\n# Improved assignment submit utility\n# - Encodes provided pandas DataFrames, PIL Images, or numpy arrays as base64 payloads\n# - Attaches the source .py or .ipynb file (must contain "_class{no}" in filename)\n# - Posts the payload to the remote submission endpoint with error handling and timeout\n\nimport base64\nimport io\nimport logging\nimport os\nfrom typing import Any, Dict, List

As you can see, the LLM suggested several improvements that I will consider for future versions of this function.